In [30]:
import pandas as pd
import json
import os



def get_merge_df(file, folder='./predictions'):
    with open(os.path.join(folder, file), 'r', encoding='utf-8') as f:
        pred_json = json.load(f)

    pred_data = [{'id': k, 'pred_answer_text': v} for k, v in pred_json.items()]
    pred_df = pd.DataFrame(pred_data)

    answer_df = pd.read_csv('./test_dataset_with_answer.csv')

    merge_df = pd.merge(pred_df, answer_df, how='inner', on='id')
    merge_df = merge_df.loc[merge_df['answer_text'] != 'none', :]

    return merge_df


def clean_text(text):
    #text = text.strip()
    return text

def calc_em(file):
    """
    실제 정답이 여러 개인 경우 예측 정답이 실제 정답 중 하나라도 포함되면 맞춘 것으로 평가 진행
    em이 높게 나오는 경향이 있음
    """
    merge_df = get_merge_df(file)

    correct_result = []
    for i, row in merge_df.iterrows():
        pred_answer = clean_text(row['pred_answer_text'])
        ground_truth = [clean_text(text) for  text in eval(row['answer_text'])]
        correct_result.append(pred_answer in ground_truth)

    em = sum(correct_result)/len(correct_result) * 100

    return f"{em:.3f}"

def calc_em_2(file):
    """
    실제 정답이 한 개인 경우에 대해서만 평가 진행
    calc_em 보다는 em이 낮게 나옴
    실제 제출 시의 em보다는 2~7 정도 높게 나옴
    """
    merge_df = get_merge_df(file)
    merge_df = merge_df.loc[merge_df['answer_cnt'] == 1, :]
    
    correct_result = []
    for i, row in merge_df.iterrows():
        pred_answer = clean_text(row['pred_answer_text'])
        ground_truth = [clean_text(text) for  text in eval(row['answer_text'])]
        correct_result.append(pred_answer in ground_truth)

    em = sum(correct_result)/len(correct_result) * 100

    return f"{em:.3f}"

In [17]:
calc_em('predictions_60.830_권혜빈.json')

'69.053'

In [18]:
calc_em('predictions_60.830_김제우.json')

'67.368'

In [19]:
calc_em('predictions_60.000_김민진.json')

'66.105'

In [26]:
calc_em_2('predictions_60.830_권혜빈.json')

'67.069'

In [27]:
calc_em_2('predictions_60.830_김제우.json')

'64.955'

In [25]:
calc_em_2('predictions_60.000_김민진.json')

'62.236'

In [31]:
calc_em_2('predictions_59.580_권혜빈.json')

'63.142'